In [9]:
import pandas as pd

# Load CSV file

url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)

#Look at the first few rows
df.head()

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
0,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,IE0,Ireland,Number,5149139
1,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1492-13a3-e055-000000000001,Carlow County Council,Number,61968
2,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-1433-13a3-e055-000000000001,Dublin City Council,Number,592713
3,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-149f-13a3-e055-000000000001,Dún Laoghaire Rathdown County Council,Number,233860
4,FY006AC01,Population,2022,2022,-,Both sexes,-,All ages,2ae19629-14a0-13a3-e055-000000000001,Fingal County Council,Number,330506


In [10]:
# Data-cleaning approach above is adapted from the lecturer’s "Weighted Stats" notebook
# Remove the "All ages" rows
# These rows cannot be used for weighted age calcuations
df = df[df["Single Year of Age"] != "All ages"]

# Keep only the male and female rows
# This dataset also includes both sexes which arent needed
df = df[df["Sex"].isin(["Male", "Female"])]

# Clean the age labels similar to notebook with counties
# Replace 'Under 1 year' with '0'
df["Single Year of Age"] = df["Single Year of Age"].str.replace("Under 1 year", "0")

# Remove all non-digit characters so '100 years and over' becomes '100'
df["Single Year of Age"] = df["Single Year of Age"].str.replace(r"\D", "", regex=True)

# Convert age and VALUE to proper integers
df["Single Year of Age"] = df["Single Year of Age"].astype("int64")
df["VALUE"] = df["VALUE"].astype("int64")

# Check the data
df.head()
df.dtypes

STATISTIC                  object
Statistic Label            object
TLIST(A1)                   int64
CensusYear                  int64
C02199V02655               object
Sex                        object
C02076V03371               object
Single Year of Age          int64
C03789V04537               object
Administrative Counties    object
UNIT                       object
VALUE                       int64
dtype: object

In [11]:
# Check the column names to see what can be used
df.columns

Index(['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'CensusYear',
       'C02199V02655', 'Sex', 'C02076V03371', 'Single Year of Age',
       'C03789V04537', 'Administrative Counties', 'UNIT', 'VALUE'],
      dtype='object')

In [12]:
# Look at the unique values in the 'SEX' column
df["Sex"].unique()

array(['Male', 'Female'], dtype=object)

In [13]:
# Select only males
males = df[df["Sex"] == "Male"]

# Total male population
total_males = males["VALUE"].sum()

# Sum of age and population for males
male_age_times_pop = (males["Single Year of Age"] * males["VALUE"]).sum()

# Weighted mean age for males
male_weighted_mean_age = male_age_times_pop / total_males

male_weighted_mean_age

np.float64(37.7394477371039)

In [14]:
# Select only females
females = df[df["Sex"] == "Female"]

# Total female population
total_females = females["VALUE"].sum()

# Sum of age and population for females
female_age_times_pop = (females["Single Year of Age"] * females["VALUE"]).sum()

# Weighted mean age for females
female_weighted_mean_age = female_age_times_pop / total_females

female_weighted_mean_age

np.float64(38.9397958987787)

In [15]:
male_by_age = males.groupby("Single Year of Age")["VALUE"].sum()
females_by_age = females.groupby("Single Year of Age")["VALUE"].sum()

difference_by_age = male_by_age - females_by_age
difference_by_age

Single Year of Age
0      2848
1      2660
2      2524
3      3036
4      3734
       ... 
96    -1258
97    -1030
98     -724
99     -462
100    -860
Name: VALUE, Length: 101, dtype: int64

## References

**CSO PxStat API – FY006A Population Dataset**  
Source of the population data used in this notebook.  
https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en  

**Pandas Documentation – read_csv**  
Used to load the CSO dataset.  
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html  

**Pandas Documentation – String Methods (.str.replace, regex)**  
Used to clean age labels such as "Under 1 year" and "100 years and over".  
https://pandas.pydata.org/docs/user_guide/text.html  

**Pandas Documentation – Boolean Indexing**  
Used to select rows for "Male" and "Female".  
https://pandas.pydata.org/docs/user_guide/indexing.html#boolean-indexing  

**Pandas Documentation – astype**  
Used to convert cleaned age values and population counts to integers.  
https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html  

**Pandas Documentation – groupby**  
Used to group population counts by age for each sex.  
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html  

**NumPy Weighted Average Formula (lecturer weighted stats notebook)**  
Used indirectly when calculating weighted mean age:  
weighted mean = sum(age × population) / sum(population)  
https://numpy.org/doc/stable/reference/generated/numpy.average.html  

**Lecturer "Weighted Stats" Notebook**  
Cleaning logic for age labels and numeric conversion adapted from lecture examples.